# Data Quality Analysis

The data set is not presented in way that would pass validation standards for USPS or email format, this is simply information with which to work.

The analysis presented here has three purposes: 

- Create metrics that show the health* of addresses, phones, and emails for all living individuals in the data set.
- Break that analysis out into segments by entity type to show the health* of each entity category.
- Note any potential relationships and any data anomalies.

*Health indicates the number of instances where the information is valid.

The analysis will be carried out using Python. 

I will begin by impoting the necessary `pandas` library to work with data sets and load the data set into our working environment. 

In [293]:
# Import pandas library

import pandas as pd 


# Load data set into working environment

file_name = "DQS_dataset.xlsx"   # save file name as an object

dqs_data = pd.read_excel(file_name)  # read data into working environment 

## Data Exploration

Now that we have our data set in our working environment I will begin exploring the data. 

In [294]:
# Explore Column Names

dqs_data.columns


Index(['ID', 'FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME', 'AGE', 'DECEASED_FLAG',
       'ENTITY_TYPE', 'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE',
       'ZIP', 'BAD_ADDR_FLG', 'EMAIL', 'BAD_EMAIL_FLG', 'PHONE',
       'BAD_PHONE_FLG'],
      dtype='object')

These are the rows I will work with to conduct our analysis. 

I will continue by checking the number of rows and columns.

In [295]:
# Dimensions of the DataFrame

dqs_data.shape 


(120, 17)

There are 120 rows and 17 columns in the data set. 

I will continue to explore the number of entries in each column and the type of data found within each column.

In [296]:
# Information about each variable, details 

dqs_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 17 columns):
ID                120 non-null int64
FIRST_NAME        120 non-null object
MIDDLE_NAME       63 non-null object
LAST_NAME         120 non-null object
AGE               81 non-null float64
DECEASED_FLAG     4 non-null object
ENTITY_TYPE       120 non-null object
ADDRESS_LINE_1    120 non-null object
ADDRESS_LINE_2    27 non-null object
CITY              120 non-null object
STATE             119 non-null object
ZIP               116 non-null float64
BAD_ADDR_FLG      6 non-null object
EMAIL             118 non-null object
BAD_EMAIL_FLG     5 non-null object
PHONE             102 non-null float64
BAD_PHONE_FLG     3 non-null object
dtypes: float64(3), int64(1), object(13)
memory usage: 16.0+ KB


Here we can see that a complete column is made up of 120 entries, some columns with less than 120 entries denote missing information, but others are flags raised when the information contained in that particular row is bad. 

## Data Cleaning

As mentioned in the beginning, we are only to analyze the health of addresses, phones, and emails for all living individuals. Let's begin by taking out the individuals who are deceased. I will use the `DECEASED_FLAG` column to perform the operation. By looking at the info from our columns, found above,  we that there are four rows flagged as deceased. 

To make taking out of rows of people who are deceased easier, I will assign a value of 0 to those who are not deceased and a value of 1 to those who are.  

The operation will take out the 4 rows that are flagged as being deceased. 

In [297]:
# Assign value of 0 to those not deceased(in our case NaN = not deceased), and value of 1 to those who are

dqs_data['DECEASED_FLAG'] = dqs_data['DECEASED_FLAG'].fillna(value = 0)  # replace NaN with 0

dqs_data['DECEASED_FLAG'] = dqs_data['DECEASED_FLAG'].replace('X', 1)  # replace X with 1


# Now we only keep rows where 'DECEASED_FLAG' = 0, not deceased

dqs_data = dqs_data.loc[dqs_data['DECEASED_FLAG'] == 0]

Now that we have taken out the rows flagged as deceased, let's check out our new number of rows and columns.

In [298]:
# Print new number of rows and columns

dqs_data.shape

(116, 17)

We now have 116 rows, previously 120. We can now continue with testing the quality of our data. 

## Data Quality

Data quality refers to the ability of a set of data to serve an intended purpose. Low-quality data cannot be used effectively to provide the insights you are looking for. 

According to [Syncsort](https://blog.syncsort.com/2018/02/data-quality/how-to-measure-data-quality-7-metrics/), there are seven data quality metrics to measure the quality of your data. There are various articles which can be helpful in measuring data quality and what metrics, but they all revolve around the same basic metrics. 

Given the data that we have available in our dataset, there are three metrics I will use:

- Number of missing values
- Incorrect data
- Ratio of data to errors

Let's begin by assesing the quality of the addresses in our data, continued by email and phone numbers. 

### Quality/Validity of Addresses

The columns that will help us determine the quality are `ADDRESS_LINE_1`, `ADDRESS_LINE_2`, `CITY`, `STATE`, `ZIP`, and `BAD_ADDR_FLG`. 

I will create a dataset `dqs_addr` to inspect the quality/validity of the address portion of the data and begin by taking a look at the number of missing values and then proceed to inspect the accuracy of the data. 

In [299]:
# Create dqs_addr dataset

dqs_addr = dqs_data[['ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIP', 'BAD_ADDR_FLG']]


# Find the total number of missing values by column

dqs_addr.isnull().sum()

ADDRESS_LINE_1      0
ADDRESS_LINE_2     89
CITY                0
STATE               1
ZIP                 4
BAD_ADDR_FLG      110
dtype: int64

We can see that the columns `ADDRESS_LINE_2` and `BAD_ADDR_FLG` have many missing values, but we shouldn't consider this as true missing data. Not everyone has a second address line (such as apartment or suite number), and missing values in the column flagging a bad address simply means that it is not a bad address. 

The columns we should consider as having missing values are `STATE` with 1 and `ZIP` with 4 missing values. We can explore these rows with missing values further to understand why they might be missing. 

For this I will create a dataset `null_s_z` to include only rows with missing values in `STATE` and `ZIP` for further inspection. 

In [300]:
# Dataset for rows with missing values in address section

null_s_z = dqs_addr[dqs_addr[['STATE', 'ZIP']].isnull().any(axis=1)]

print(null_s_z)

                            ADDRESS_LINE_1 ADDRESS_LINE_2     CITY      STATE  \
16  No.609 Youhao Street, Jinzhou District            NaN   DALIAN        NaN   
45            D65, Sispal Vihar Sohna Road      Sector 49  GURGAON    HARYANA   
46                      D22 jinxiu Huayuan            NaN  CHIFENG  NEIMENGGU   
82                      D22 jinxiu Huayuan            NaN  CHIFENG  NEIMENGGU   

    ZIP BAD_ADDR_FLG  
16  NaN          NaN  
45  NaN          NaN  
46  NaN          NaN  
82  NaN          NaN  


We see that all the rows containing missing values for `STATE` and `ZIP` are for people who live outside the US, it is undestandable that addresses for different countries are formatted differently and are not necessarily missing data. Addresses outside of the US could be flagged or put in another dataset altogether to be analyzed on there own. 

We will continue to asses the validity of our data by analyzing the accuracy of the data found in the addresses. 

Our dataset contains a column `BAD_ADDR_FLG` which denotes an address that is wrong for that particular person. 

In [264]:
# Count number of columns with a bad address

dqs_addr['BAD_ADDR_FLG'].count()

6

As we can see from counting the number of bad address flag, we already know that we have six addresses that are bad. 

Let's continue by analyzing the data containted in each row of the `ADDRESS_LINE_1` column to make sure we actually have an address in the row. To do this I will run the through the dataset checking if each value in the `ADDRESS_LINE_1` column starts with a number, as do all addresses in the US. I create a new column `good_addr` which will be True if it does start with a number and False if not.

I will then isolate those that do not start with a number to analyze further. 

In [ ]:
# Create new column good_addr to denote addresses starting with a number

dqs_addr['good_addr'] = dqs_addr['ADDRESS_LINE_1'].str[0].str.isdigit()


# Isolate rows which address doesn't start with a number

bad_addr = dqs_addr[dqs_addr['good_addr'] == False]

Now that we have isolated the rows with addresses not starting with numbers; let's see how many we have.

In [302]:
# Number of rows in bad_addr data set

print(bad_addr.shape)

(10, 7)


We have a total of 10 rows with bad addresses, let's explore these further. 

In [303]:
# Display the rows with bad addresses

print(bad_addr)

                             ADDRESS_LINE_1  \
11        The University of Texas at Austin   
16   No.609 Youhao Street, Jinzhou District   
32        The University of Texas at Austin   
45             D65, Sispal Vihar Sohna Road   
46                       D22 jinxiu Huayuan   
48        The University of Texas at Austin   
82                       D22 jinxiu Huayuan   
88        The University of Texas at Austin   
103                        P. O. Box 276177   
111                     .527 Stowers Avenue   

                           ADDRESS_LINE_2      CITY      STATE      ZIP  \
11       204 E Dean Keeton St, Stop C2200    Austin         TX  78712.0   
16                                    NaN    DALIAN        NaN      NaN   
32   2406 Robert Dedman Drive, Stop E3100    Austin         TX  78712.0   
45                              Sector 49   GURGAON    HARYANA      NaN   
46                                    NaN   CHIFENG  NEIMENGGU      NaN   
48              2101 Speedway, S

We can see that four rows have The University of Texas at Austin instead of an address, four rows have addresses outside the US calling for a different format, one row is a P.O. Box, and one row starts with a decimal point. 

From the overall analysis of the address portion of the data set, I would say we have a healthy dataset. We do we have 10 rows that present information that isn't valid, but of those ten, four are addresses outside of the US that should be flagged in some way or formatted differently so they don't present as invalid information. We have four rows with the address listed as the university, in this case there should be constraints applied so when the user is inputting the data it has to begin with a number and be valid address and not a location name. The same restriction, address beginning with a number, could be applied to the rows containin a P.O. Box as an address and an address starting with a decimal point. 

Given the results we find ourselves with a dataset that has about 90% of addresses being valid, with solutions to increase the validity noted above.

### Quality/Validity of Emails and Phone Numbers

We can now continue with the next task of analyzing the health of emails and phone number in our data. 

For this we will analyze the `EMAIL`, `BAD_EMAIL_FLG`, `PHONE`, and `BAD_PHONE_FLG` columns in new data set `dqs_em_ph`, and begin by analyzing the number of missing values.

In [304]:
# Create dqs_em_ph data set

dqs_em_ph = dqs_data[['EMAIL', 'BAD_EMAIL_FLG', 'PHONE', 'BAD_PHONE_FLG']]

# Number of missing values

# Find the total number of missing values by column

dqs_em_ph.isnull().sum()

EMAIL              2
BAD_EMAIL_FLG    111
PHONE             16
BAD_PHONE_FLG    113
dtype: int64

We can see that we have some missing data, but as mentioned before, columns `BAD_EMAIL_FLG` and `BAD_PHONE_FLG` don't denote missing data just that the data contained in that row isn't bad. In the columns `EMAIL` and `PHONE` we do see 2 and 16 missing values respectively. 

Given a total of 116 observations, that leaves us with 98% valid emails and 86% valid phone numbers, with 5 emails and 3 phone number flagged as no longer working. 

## Quality/Validity by Entity Type

In our dataset we have a column `ENTITY_TYPE`, detailing they type of each individual. We analyze the health of each of these entities. We will begin by splitting the dataset into its respective entity type and analyzing the health of addresses, emails, and phone numbers. 

In [305]:
# Unique entries in ENTITY_TYPE

dqs_data.ENTITY_TYPE.unique()

array(['FRIEND', 'ALUMNI', 'PARENT'], dtype=object)

We see that each individual is either a friend, alumni, or parent. Let's separate each group and analyze them for data health.

In [308]:
# Creat three different data sets by ENTITY_TYPE

dqs_friend = dqs_data[dqs_data['ENTITY_TYPE'] == 'FRIEND']

dqs_alum = dqs_data[dqs_data['ENTITY_TYPE'] == 'ALUMNI']

dqs_parent = dqs_data[dqs_data['ENTITY_TYPE'] == 'PARENT']


# Separate our rows of interest

dqs_friend = dqs_friend[['ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIP', 'BAD_ADDR_FLG',
                         'EMAIL', 'BAD_EMAIL_FLG', 'PHONE', 'BAD_PHONE_FLG']]

dqs_alum = dqs_alum[['ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIP', 'BAD_ADDR_FLG',
                     'EMAIL', 'BAD_EMAIL_FLG', 'PHONE', 'BAD_PHONE_FLG']]

dqs_parent = dqs_parent[['ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIP', 'BAD_ADDR_FLG',
                         'EMAIL', 'BAD_EMAIL_FLG', 'PHONE', 'BAD_PHONE_FLG']]

Now that we have split the data set by `ENTITY_TYPE` we can anlyze each for data health.

### Friend Entity Type

We will begin by analyzing the friend entity type for data quality/health.

In [311]:
# Observations in FRIEND ENTITY_TYPE

dqs_friend.shape

(63, 10)

In [310]:
# Number of missing values in FRIEND ENTITY_TYPE

dqs_friend.isnull().sum()

ADDRESS_LINE_1     0
ADDRESS_LINE_2    52
CITY               0
STATE              1
ZIP                4
BAD_ADDR_FLG      61
EMAIL              2
BAD_EMAIL_FLG     61
PHONE              7
BAD_PHONE_FLG     61
dtype: int64

We have a total of 63 observations with no missing addresses, one missing state, four missing zips, two missing emails and seven missing phones. With flags for two bad addresses, two bad emails, and two bad phone numbers. 

Missing values for `STATE` and `ZIP` are part of the data we encountered before where the address was out of the country. 

Now let's check the validity of `ADDRESS_LINE_1` as we did before to check if addresses begin with numbers. 

In [313]:
# Create new column good_addr to denote addresses starting with a number

dqs_friend['good_addr'] = dqs_friend['ADDRESS_LINE_1'].str[0].str.isdigit()


# Isolate rows which address doesn't start with a number

bad_addr_fr = dqs_friend[dqs_friend['good_addr'] == False]

# Print number of rows have bad address in dqs_friend

print(bad_addr_fr.shape)

(9, 11)


We see that in the friend entity type, we have 9 out of the 10 rows from our total data set with invalid addresses. 

### Alum Entity Type

In [314]:
# Observations in ALUMNI ENTITY_TYPE

dqs_alum.shape

(37, 10)

In [315]:
# Number of missing values in ALUMNI ENTITY_TYPE

dqs_alum.isnull().sum()

ADDRESS_LINE_1     0
ADDRESS_LINE_2    25
CITY               0
STATE              0
ZIP                0
BAD_ADDR_FLG      34
EMAIL              0
BAD_EMAIL_FLG     35
PHONE              1
BAD_PHONE_FLG     36
dtype: int64

We have a total of 37 observations with only one missing phone number, and flags for three bad addresses, two bad emails, and one bad phone number.

In [318]:
# Create new column good_addr to denote addresses starting with a number

dqs_alum['good_addr'] = dqs_alum['ADDRESS_LINE_1'].str[0].str.isdigit()


# Isolate rows which address doesn't start with a number

bad_addr_al = dqs_alum[dqs_alum['good_addr'] == False]

# Print number of rows have bad address in dqs_friend

print(bad_addr_al.shape)

(1, 11)


In the alumni entity type we find one row with an address not beginning with a number, a much more healhty data set when compared to the friend entity type.

### Parent Entity Type

In [319]:
# Observations in PARENT ENTITY_TYPE

dqs_parent.shape

(16, 10)

In [320]:
# Number of missing values in ALUMNI ENTITY_TYPE

dqs_parent.isnull().sum()

ADDRESS_LINE_1     0
ADDRESS_LINE_2    12
CITY               0
STATE              0
ZIP                0
BAD_ADDR_FLG      15
EMAIL              0
BAD_EMAIL_FLG     15
PHONE              8
BAD_PHONE_FLG     16
dtype: int64

We have a total of 16 observations with eight missing phone numbers, one bad address flag, and one bad email flag. 

In [322]:
# Create new column good_addr to denote addresses starting with a number

dqs_parent['good_addr'] = dqs_parent['ADDRESS_LINE_1'].str[0].str.isdigit()


# Isolate rows which address doesn't start with a number

bad_addr_pr = dqs_parent[dqs_parent['good_addr'] == False]

# Print number of rows have bad address in dqs_friend

print(bad_addr_pr.shape)

(0, 11)


In the parent entity type we find zero invalid addresses. Making it the healthiest when compared to the friend and alumni entity types. With the friend entity type being the least healthy. 

## Data Anomalies

After analysis we concluded that just around 90% of our data is valid/healthy data, and the parent entity type is the group with the healthiest and the friend entity type with least healthy data. 

One more thing I would like to note that affects data quality, is the data types found within our data. Our data is made up of letters, words, and numbers. An important aspect of quality data is having it labeled as the correct type to make analyzing the data easier. 

Letters and words are known as string data, and numbers can be either integers (whole numbers) like age, or floats for numbers with decimal places. At the begining of our analysis, during the data exploration we saw that a lot of columns are read in as objects instead of strings or numbers. This can cause some operations to not function properly. Moreover columns like addresses and phone numbers should be labeled as strings given that mathematical operations with these columns are not possible. Setting restrictions as to the data type of each column when building out the database, will allow for an easier analys of the data. 